# **Import Modules and Libraries**

## *Python*

In [1]:
#import 
import cx_Oracle
import pandas as pd
import os
import sys
import random
import string

## *Custom*

In [2]:
#import customised modules

#define path
sys.path.append(os.path.abspath(os.path.join('..')))

##import relevant modules
from scripts.oracle_db import process_df_for_db

# **Save CSV file to Database**

In [3]:
#establish connection to Oracle Database
db_password = os.environ.get('DB_PASSWORD')
conn = cx_Oracle.connect("SYS", db_password, "localhost", mode=cx_Oracle.SYSDBA)
cursor = conn.cursor()

In [4]:
#define df path
df_path = os.path.join(os.path.dirname(os.getcwd()), 'data/all_preprocessed_reviews.csv')

#call function from module
df = process_df_for_db (df_path)
df

,review_id,bank_id,bank_name,rating,review_date,review_text,source
0,947086,1,Abyssinia,3,2025-06-05,it's not working,Google Play
1,503050,1,Abyssinia,1,2025-06-03,"Hello, I’m facing a problem with the BOA Mobil...",Google Play
2,562871,1,Abyssinia,5,2025-06-03,exceptional,Google Play
3,999965,1,Abyssinia,5,2025-06-02,BoA Mobile good bank,Google Play
4,765754,1,Abyssinia,1,2025-06-01,this is worest app 24/7 loading,Google Play
...,...,...,...,...,...,...,...
1425,335368,3,Dashen,5,2025-01-13,Best,Google Play
1426,781399,3,Dashen,5,2025-01-13,"Waw Great and innovated,user friendly, always ...",Google Play
1427,202010,3,Dashen,5,2025-01-13,It's Best waww 🙏,Google Play
1428,347310,3,Dashen,5,2025-01-13,Always one step ahead,Google Play


In [ ]:
#save df to csv
insert_query = """
    INSERT INTO reviews (review_id, bank_id, bank_name, rating, review_date, review_text, source) 
    VALUES (:1, :2, :3, :4, TO_DATE(:5, 'YYYY-MM-DD'), :6, :7)
"""

for _, row in df.iterrows():
    cursor.execute(insert_query, (row['review_id'],row['bank_id'], row['bank_name'], 
                                   row['rating'], row['review_date'],
                                  row['review_text'], row['source']))

conn.commit()  #save changes
print("Data inserted successfully!")

In [6]:
#close the connection
cursor.close()
conn.close()